In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import json
from shapely.geometry import Polygon
import urllib.request

In [15]:
def create_geomask(input_file):
    if isinstance(input_file, gpd.GeoDataFrame):
        input_gdf = input_file
    else:
        input_gdf = gpd.read_file(input_file) 
    input_gdf.set_crs(4326, allow_override=True)
    bounding_box = Polygon([ (-180, -90), (180, -90), (180, 90), (-180, 90) ])
    inverse_polygon = bounding_box.difference(input_gdf.geometry.iloc[0])
    inverse_gdf = gpd.GeoDataFrame(geometry=[inverse_polygon], crs=input_gdf.crs)
    return inverse_gdf

In [2]:
# make a list of all files in the directory
files = os.listdir('./')
# make a list called poly_files that includes files that include the string poly
poly_files = [f for f in files if 'poly' in f]
point_files = [f for f in files if 'point' in f]

In [34]:
poly_files

['MLP64-16poultryMSpolysEnsemble7of10models2023-10-22-23.geojson',
 'MLP64-16poultrySCpolysEnsemble7of10models2023-10-23.geojson',
 'MLP64-16poultryThresh0.75grid160ALpolysEnsemble6of8models2023-10-23.geojson',
 'MLP64-16poultryThresh0.7grid160ARpolysMLP64-16poultry2023-10-22T18_36.joblib.geojson',
 'MLP64-16poultryThresh0.7grid160NCpolysMLP64-16poultry2023-10-22T13_44.joblib.geojson',
 'MLP64-16poultryThresh0.75grid160GApolysEnsemble6of8models2023-10-23.geojson']

In [43]:
test_poly = gpd.read_file(poly_files[5])
test_poly.iloc[1].explore()

AttributeError: 'Series' object has no attribute 'explore'

In [3]:
gdfs = [gpd.read_file(f) for f in poly_files]
poly_gdf = pd.concat(gdfs, ignore_index=True)

In [4]:
poly_gdf

,level_0,level_1,geometry,confidence
0,0.0,0.0,"POLYGON ((-90.56735 31.01618, -90.56750 31.016...",NaN
1,0.0,1.0,"POLYGON ((-90.91016 31.05532, -90.91032 31.055...",NaN
2,0.0,2.0,"POLYGON ((-90.59620 31.07439, -90.59636 31.074...",NaN
3,0.0,3.0,"POLYGON ((-90.64815 31.07534, -90.64831 31.075...",NaN
4,0.0,4.0,"POLYGON ((-90.58939 31.07859, -90.58955 31.078...",NaN
...,...,...,...,...
16144,NaN,NaN,"POLYGON ((-81.84641 32.34506, -81.84658 32.345...",0.988282
16145,NaN,NaN,"POLYGON ((-81.66657 32.39666, -81.66673 32.396...",0.840155
16146,NaN,NaN,"POLYGON ((-81.93349 32.40825, -81.93361 32.408...",0.942303
16147,NaN,NaN,"POLYGON ((-82.00699 32.41316, -82.00716 32.413...",0.916321


In [5]:
gdfs = [gpd.read_file(f) for f in point_files]
point_gdf = pd.concat(gdfs, ignore_index=True)
point_gdf

,confidence,geometry,level_0,level_1
0,0.964746,POINT (-94.34613 33.66657),NaN,NaN
1,0.942335,POINT (-94.35067 33.68095),NaN,NaN
2,0.891230,POINT (-94.34125 33.68572),NaN,NaN
3,0.924113,POINT (-94.33871 33.68902),NaN,NaN
4,0.861294,POINT (-94.35912 33.69745),NaN,NaN
...,...,...,...,...
16144,NaN,POINT (-88.47301 34.42117),0.0,1543.0
16145,NaN,POINT (-88.53258 34.44136),0.0,1544.0
16146,NaN,POINT (-88.53213 34.45053),0.0,1545.0
16147,NaN,POINT (-88.21684 34.46439),0.0,1546.0


In [44]:
# return a random point gdf row
point_gdf.sample()

,confidence,geometry,level_0,level_1
15738,NaN,POINT (-89.16045 32.01081),0.0,1137.0


In [50]:
# return the row from the poly_gdf that matches 	1137.0 in level_1 column
poly_gdf[poly_gdf['level_1'] == 1137.0]

,level_0,level_1,geometry,confidence
1137,0.0,1137.0,"POLYGON ((-89.15989 32.00800, -89.16005 32.007...",NaN


In [7]:
poly_gdf.to_file('poly-0102202024.geojson', driver='GeoJSON')
point_gdf.to_file('points-0102202024.geojson', driver='GeoJSON')

In [20]:
states_files = os.listdir('./states')
gdfs = [gpd.read_file('./states/'+f) for f in states_files]
states_gdf = pd.concat(gdfs, ignore_index=True)
states_gdf

,id,STUSPS,NAME,ALAND,AWATER,geometry
0,GA,GA,Georgia,1.489635e+11,4.947080e+09,"MULTIPOLYGON Z (((-81.27939 31.30792 0.00000, ..."
1,SC,SC,South Carolina,7.785791e+10,5.074749e+09,"MULTIPOLYGON Z (((-79.50795 33.02008 0.00000, ..."
2,AL,AL,Alabama,1.311724e+11,4.594951e+09,"MULTIPOLYGON Z (((-88.05338 30.50699 0.00000, ..."
3,MS,MS,Mississippi,1.215319e+11,3.928588e+09,"MULTIPOLYGON Z (((-88.50297 30.21523 0.00000, ..."
4,AR,AR,Arkansas,1.347716e+11,2.960201e+09,"POLYGON Z ((-94.61783 36.49941 0.00000, -94.36..."
5,NC,NC,North Carolina,1.259180e+11,1.347272e+10,"MULTIPOLYGON Z (((-75.72681 35.93584 0.00000, ..."


In [21]:
# join states_gdf as one shape
states_gdf['geometry'] = states_gdf.unary_union
states_gdf

,id,STUSPS,NAME,ALAND,AWATER,geometry
0,GA,GA,Georgia,1.489635e+11,4.947080e+09,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."
1,SC,SC,South Carolina,7.785791e+10,5.074749e+09,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."
2,AL,AL,Alabama,1.311724e+11,4.594951e+09,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."
3,MS,MS,Mississippi,1.215319e+11,3.928588e+09,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."
4,AR,AR,Arkansas,1.347716e+11,2.960201e+09,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."
5,NC,NC,North Carolina,1.259180e+11,1.347272e+10,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."


In [28]:
# create a geodataframe from the first row of states_gdf
states_poly = gpd.GeoDataFrame(states_gdf.iloc[0])
states_poly = states_poly.T
states_poly

,id,STUSPS,NAME,ALAND,AWATER,geometry
0,GA,GA,Georgia,148963503399.0,4947080103.0,"MULTIPOLYGON Z (((-88.43033 30.20855 0.00000, ..."


In [29]:
states_poly.explore()

In [14]:
states_gdf.to_file('states.geojson', driver='GeoJSON')

In [30]:
create_geomask(states_poly).to_file('states-mask.geojson', driver='GeoJSON')

In [17]:
def fetch_static_map(lon, lat, filename):
    accessToken = "pk.eyJ1IjoicGxvdGxpbmUiLCJhIjoiY2xmOGo1NW4wMGVtNzNya2UyNnllZGcyciJ9.gUFn8Mj5HQbagkpQWaDqaw"
    print (str(lon), str(lat))
    static_url ="https://api.mapbox.com/styles/v1/plotline/clqybizqa00f101rj8l32czwr/static/{lon},{lat},15,0,0/500x500@2x?access_token={accessToken}&attribution=false&logo=false"
    print (static_url.format(lon=str(lon), lat=str(lat), accessToken=accessToken))
    urllib.request.urlretrieve(static_url.format(lon=str(lon), lat=str(lat), accessToken=accessToken), "./static-map-images/"+filename+".png")


In [15]:
point_data = gpd.read_file('./cafo-points-01082024.geojson')
point_data

,confidence,id,geometry
0,0.927063,AL0,POINT (-87.91460 30.23426)
1,0.871785,AL1,POINT (-88.28150 30.45140)
2,0.962882,AL2,POINT (-88.38625 31.30722)
3,0.936559,AL3,POINT (-88.42356 31.31208)
4,0.944419,AL4,POINT (-88.38711 31.49681)
...,...,...,...
16367,0.967801,SC763,POINT (-79.69362 34.74384)
16368,0.941294,SC764,POINT (-79.73538 34.77049)
16369,0.893059,SC765,POINT (-79.67360 34.78209)
16370,0.952176,SC766,POINT (-79.68223 34.78940)


In [21]:
point_data.sample().geometry.iloc[0].x

-83.23420543120203

In [32]:
# get a random row from the point_data and pass lat and lon to fetch_static_map do this 100 times with the file name being the iteration number
for i in range(100):
    row = point_data.sample()
    fetch_static_map(row.geometry.iloc[0].x, row.geometry.iloc[0].y, str(i))


-76.80847033399145 36.472369817889565
https://api.mapbox.com/styles/v1/plotline/clqybizqa00f101rj8l32czwr/static/-76.80847033399145,36.472369817889565,15,0,0/500x500@2x?access_token=pk.eyJ1IjoicGxvdGxpbmUiLCJhIjoiY2xmOGo1NW4wMGVtNzNya2UyNnllZGcyciJ9.gUFn8Mj5HQbagkpQWaDqaw&attribution=false&logo=false
-93.74701585647415 33.734418551197464
https://api.mapbox.com/styles/v1/plotline/clqybizqa00f101rj8l32czwr/static/-93.74701585647415,33.734418551197464,15,0,0/500x500@2x?access_token=pk.eyJ1IjoicGxvdGxpbmUiLCJhIjoiY2xmOGo1NW4wMGVtNzNya2UyNnllZGcyciJ9.gUFn8Mj5HQbagkpQWaDqaw&attribution=false&logo=false
-86.34124576317689 33.14179829620302
https://api.mapbox.com/styles/v1/plotline/clqybizqa00f101rj8l32czwr/static/-86.34124576317689,33.14179829620302,15,0,0/500x500@2x?access_token=pk.eyJ1IjoicGxvdGxpbmUiLCJhIjoiY2xmOGo1NW4wMGVtNzNya2UyNnllZGcyciJ9.gUFn8Mj5HQbagkpQWaDqaw&attribution=false&logo=false
-89.59658236242868 31.788173677109118
https://api.mapbox.com/styles/v1/plotline/clqybizqa00f101r